# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import pickle
import cv2
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

In [3]:
dimension = (28,28)
input_shape = (28,28,1)

# Data Augmentation

In [21]:
def augmentation(X_train, y_train, X_Test, y_test):
    
    generator = ImageDataGenerator(rotation_range=10, horizontal_flip=True, fill_mode="nearest")
    train_batches = generator.flow(X_train, y_train, batch_size=128)
#     test_batches = generator.flow(X_Test, y_test, batch_size=256)
    
    return train_batches

# Build Neural Network

In [5]:
def build_network(inputs):
    
    '''--- Simple CNN Architecture ---'''
    
    # 2D Convolutional layer to extract features by applying filter on the given image
    cnn = Sequential()
    cnn.add(Conv2D(64, (3, 3), activation='relu', input_shape=inputs))
    
    # Max Pooling layer to reduce dimensionality and Dropout layer to prevent the model from overfitting
    cnn.add(MaxPooling2D(pool_size=(2,2)))
    cnn.add(Dropout(0.3))
    
    # Flatten the current Output produced by previous layers
    cnn.add(Flatten())
    cnn.add(Dense(128, activation="relu"))
    cnn.add(Dropout(0.25))
    
    # Dense layer to give 4 outputs as 4-class classification
    cnn.add(Dense(4, activation="softmax"))
    
    return cnn

# Read Dataset

In [6]:
with open('train_image.pkl', 'rb') as f:
    train_data = pickle.load(f)
    f.close()
with open('train_label.pkl', 'rb') as f:
    train_label = pickle.load(f)
    f.close()
with open('test_image.pkl', 'rb') as f:
    test_data = pickle.load(f)
    f.close()

# Convert it into Dataframe

In [7]:
train_data = pd.DataFrame(train_data)
tests = pd.DataFrame(test_data)
train_label = pd.DataFrame(train_label)
train = pd.concat([train_data, train_label], axis=1, ignore_index=True).reset_index(drop=True)

In [8]:
# Replacing label 6 as 1 for simplicity of output
uniq  = np.unique(train_label)
train_lbl = train_label.replace(train_label[train_label[:]==6], 1)

# Validation Set

In [9]:
# Categorize the output labels and Train-Validation Sets are generated!
y = LabelBinarizer().fit_transform(train_lbl)
X_trains, X_tests, y_train, y_test = train_test_split(train_data, y, test_size=0.2, random_state=13)

# Reshaping each array into image so it can fit in the CNN!

In [10]:
X_train = X_trains.values
X_test = X_tests.values
X_train = X_train.reshape(X_train.shape[0], dimension[0], dimension[1],1)
X_test = X_test.reshape(X_test.shape[0], dimension[0], dimension[1],1)

In [12]:
# Normalize
test = tests.values.reshape(tests.shape[0], dimension[0], dimension[1], 1)
test = test/255.0

# Normalization of all the images

In [14]:
X_train = X_train/255.0
X_test = X_test/255.0

## Augment the train Data

In [22]:
train_batch = augmentation(X_train, y_train, X_test, y_test)

# build the model

In [16]:
cnn = build_network(input_shape)
cnn.summary()
opt = Adam(lr=1e-3, decay=1e-3 / 200)
cnn.compile(metrics=["accuracy"], optimizer = opt, loss= "categorical_crossentropy")

Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 10816)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1384576   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
den

## Model is trained on both types of data: Augmented and Without Augmented

In [17]:
model = cnn.fit(X_train, y_train, batch_size=256, verbose=1, epochs=20, validation_data=(X_test, y_test))

Train on 6400 samples, validate on 1600 samples
Epoch 1/20
6400/6400 [==============================] - 15s 2ms/step - loss: 1.3772 - acc: 0.3341 - val_loss: 1.3606 - val_acc: 0.4738
Epoch 2/20
6400/6400 [==============================] - 14s 2ms/step - loss: 1.3201 - acc: 0.4777 - val_loss: 1.2665 - val_acc: 0.4800
Epoch 3/20
6400/6400 [==============================] - 13s 2ms/step - loss: 1.1815 - acc: 0.5386 - val_loss: 1.0952 - val_acc: 0.5894
Epoch 4/20
6400/6400 [==============================] - 12s 2ms/step - loss: 1.0075 - acc: 0.6075 - val_loss: 0.9286 - val_acc: 0.6663
Epoch 5/20
6400/6400 [==============================] - 13s 2ms/step - loss: 0.8833 - acc: 0.6475 - val_loss: 0.8257 - val_acc: 0.6863
Epoch 6/20
6400/6400 [==============================] - 13s 2ms/step - loss: 0.8113 - acc: 0.6628 - val_loss: 0.7708 - val_acc: 0.6944
Epoch 7/20
6400/6400 [==============================] - 13s 2ms/step - loss: 0.7735 - acc: 0.6698 - val_loss: 0.7553 - val_acc: 0.7006
Epoch 8

In [23]:
model = cnn.fit_generator(train_batch, steps_per_epoch=6400//128, epochs=30,validation_data=(X_test, y_test), validation_steps=1600//128)

Epoch 1/30
50/50 [==============================] - 14s 286ms/step - loss: 0.6352 - acc: 0.7317 - val_loss: 0.5945 - val_acc: 0.7525
Epoch 2/30
50/50 [==============================] - 15s 293ms/step - loss: 0.6238 - acc: 0.7423 - val_loss: 0.5857 - val_acc: 0.7550
Epoch 3/30
50/50 [==============================] - 14s 286ms/step - loss: 0.6145 - acc: 0.7453 - val_loss: 0.5737 - val_acc: 0.7719
Epoch 4/30
50/50 [==============================] - 15s 290ms/step - loss: 0.6152 - acc: 0.7463 - val_loss: 0.5712 - val_acc: 0.7825
Epoch 5/30
50/50 [==============================] - 14s 276ms/step - loss: 0.5998 - acc: 0.7514 - val_loss: 0.5754 - val_acc: 0.7744
Epoch 6/30
50/50 [==============================] - 14s 275ms/step - loss: 0.6036 - acc: 0.7541 - val_loss: 0.5676 - val_acc: 0.7725
Epoch 7/30
50/50 [==============================] - 15s 294ms/step - loss: 0.5828 - acc: 0.7634 - val_loss: 0.5456 - val_acc: 0.7844
Epoch 8/30
50/50 [==============================] - 14s 275ms/step - 

# Measure Performance on the Whole Training Set

In [24]:
score = cnn.evaluate(X_train, y_train, verbose=0)
print('Training Loss:', score[0])
print('Training Accuracy:', score[1])

Training Loss: 0.46025737941265105
Training Accuracy: 0.8196875


# Predict Classes for the Test Set

In [25]:
predicted_classes = cnn.predict_classes(test)

In [26]:
# Replace with its original label i.e., 1 is replaced with 6
x = pd.DataFrame(predicted_classes)
u = x.replace(x[x[:]==1], 6)

In [27]:
# Write results into csv file
u.to_csv("Meet_Shah.csv")